In [14]:
############################################
# CUSTOMER LIFETIME VALUE SEGMENTATION ON ONLINE RETAILS DATASET
############################################


#####################################################
# DATASET
#####################################################
#This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered #non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.
## https://archive.ics.uci.edu/ml/datasets/Online+Retail+II

#Attribute Information
#Additional Information

#InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it #indicates a cancellation.
#StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
#Description: Product (item) name. Nominal.
#Quantity: The quantities of each product (item) per transaction. Numeric.
#InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
#UnitPrice: Unit price. Numeric, Product price per unit in sterling.
#CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
#Country: Country name. Nominal, the name of the country where each customer resides.



In [15]:
###############################################################
# TASK 1: DATA PREPROCESSING and INITIAL ANALYSIS
###############################################################

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Here we read the data
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2009-2010")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [16]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [17]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,525461.00000,10.33767,107.42411,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,525461.00000,4.68883,146.12691,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,417534.00000,15360.64548,1680.81132,12346.00000,13983.00000,15311.00000,16799.00000,18287.00000


In [18]:
#There was a known issue in the story of this dataset. In the Invoice part, there were statements with C at the beginning.
# These are returned purchases. And causing – (negative) values. These observations negatively affects the variables of quantity and price.
# Let's delete these negative values first.
df = df[~df["Invoice"].str.contains("C", na=False)]

In [19]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,515255.00000,10.95663,104.35401,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,515255.00000,3.95637,127.68856,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,407695.00000,15368.50411,1679.79570,12346.00000,13997.00000,15321.00000,16812.00000,18287.00000


In [20]:
# After A describe query, here we see negative values in quantity and price variable. We need to fix this problem as well
df = df[(df['Quantity'] > 0)]

In [21]:
# Lets fix price variable
df = df[(df["Price"] >0)]

In [22]:
# We need to create new variable called "TotalPrice" in order to use it in CLTV score
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [23]:
#Let's drop NULL values, our dataset is rich enough
df.dropna(inplace=True)

In [24]:
#Below is a function that calculates the invoice and total_price variables that required for calculating cltv score, for each customer . #Cltvcalculation will be made according to these results.

cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})
# Here we rename our new variable
cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

In [25]:
##################################################
# 2. Average Order Value (average_order_value = total_price / total_transaction)
##################################################

#Below, we have calculated and saved the Average Order Value for each customer, which is required for the customer value calculation.

cltv_c.head()
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]


In [26]:
##################################################
# 3. Purchase Frequency (total_transaction / total_number_of_customers)
##################################################
# Since we have listed all purchases grouped by each unique customer, now we can see Purchase Frequency variable which is needed as well
cltv_c.head()
# shape of cltv_c dataframe gives the total number of all our customers.
cltv_c.shape[0]
cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

In [27]:
cltv_c.shape

(4312, 5)

In [29]:
##################################################
# 4. Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)
##################################################

#If we take indexing shape of total transactions which indicate at least 2 transactions, that will give us the nr. of repeated customers.
# then we need to divide to nr. of  all the customers.

repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

churn_rate = 1 - repeat_rate

In [30]:
churn_rate

0.32908163265306123

In [31]:
##################################################
# 5. Profit Margin (profit_margin =  total_price * 0.10)
##################################################
#If we set Profit Margin as 0.10
cltv_c['profit_margin'] = cltv_c['total_price'] * 0.10
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin
Customer ID,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300


In [32]:
##################################################
# 6. Customer Value (customer_value = average_order_value * purchase_frequency)
##################################################
# Customer Value is calculated as below
cltv_c['customer_value'] = cltv_c['average_order_value'] * cltv_c["purchase_frequency"]

In [33]:
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value
Customer ID,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08647
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30689
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05152
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61947
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06979


In [34]:
##################################################
# 7. Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)
##################################################
# Now the cltv dataframe is ready to calculate CLTV values for each customer, Lets see top 5
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02064,34916.43500,80.97503,8591666.19527
14646.00000,78,170278,248396.50000,3184.57051,0.01809,24839.65000,57.60587,4348190.36027
14156.00000,102,108107,196566.74000,1927.12490,0.02365,19656.67400,45.58598,2722937.51052
14911.00000,205,69722,152147.57000,742.18327,0.04754,15214.75700,35.28469,1631351.87152
13694.00000,94,125893,131443.19000,1398.33181,0.02180,13144.31900,30.48311,1217569.56993


In [35]:
##################################################
# 8. Identifying New CLTV Segments
##################################################
# New 4 segments have been identified from the bottom "D" to top "A" segments 

cltv_c.sort_values(by="cltv", ascending=False).tail()

cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
12346.00000,11,70,372.86000,33.89636,0.00255,37.28600,0.08647,9.79736,C
12347.00000,2,828,1323.32000,661.66000,0.00046,132.33200,0.30689,123.40915,B
12348.00000,1,373,222.16000,222.16000,0.00023,22.21600,0.05152,3.47816,D
12349.00000,3,993,2671.14000,890.38000,0.00070,267.11400,0.61947,502.81810,A
12351.00000,1,261,300.93000,300.93000,0.00023,30.09300,0.06979,6.38188,D


In [36]:
#Lets see our top class segment called A
cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02064,34916.43500,80.97503,8591666.19527,A
14646.00000,78,170278,248396.50000,3184.57051,0.01809,24839.65000,57.60587,4348190.36027,A
14156.00000,102,108107,196566.74000,1927.12490,0.02365,19656.67400,45.58598,2722937.51052,A
14911.00000,205,69722,152147.57000,742.18327,0.04754,15214.75700,35.28469,1631351.87152,A
13694.00000,94,125893,131443.19000,1398.33181,0.02180,13144.31900,30.48311,1217569.56993,A


In [38]:
#Here we take descriptive statistics for the segments
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

total_transaction              total_unit                total_price  \
                     mean    sum count       mean      sum count        mean   
segment                                                                        
D                 1.22913   1325  1078  109.20686   117725  1078   178.63922   
C                 2.00649   2163  1078  283.47217   305583  1078   476.26545   
B                 3.76809   4062  1078  680.71614   733812  1078  1132.30296   
A                10.81911  11663  1078 4064.22449  4381234  1078  6405.74532   

                            average_order_value                     \
                  sum count                mean          sum count   
segment                                                              
D        192573.08000  1078           157.80465 170113.41550  1078   
C        513414.15300  1078           294.33285 317290.81764  1078   
B       1220622.59000  1078           390.09746 420525.05782  1078   
A       6905393.45100  1078           671.05554 723397.87680  1078   

        purchase_frequency               profit_margin                     \
                      mean     sum count          mean          sum count   
segment                                                                     
D                  0.00029 0.30728  1078      17.86392  19257.30800  1078   
C                  0.00047 0.50162  1078      47.62654  51341.41530  1078   
B                  0.00087 0.94202  1078     113.23030 122062.25900  1078   
A                  0.00251 2.70478  1078     640.57453 690539.34510  1078   

        customer_value                         cltv                       
                  mean        sum count        mean            sum count  
segment                                                                   
D              0.04143   44.65981  1078     2.65309     2860.02697  1078  
C              0.11045  119.06636  1078    16.91865    18238.30105  1078  
B              0.26259  283.07574  1078    96.35431   103869.94237  1078  
A              1.48556 1601.43633  1078 23462.60162 25292684.54322  1078